In [1]:
import os
import pickle
import dill

dill._dill._reverse_typemap['ObjectType'] = object
dill._dill._reverse_typemap[b'ListType'] = list

import time
import numpy as np
import collections

class GlobalTimer:
    def __init__(self, scale=1000, timeunit='ms'):
        self.reset(scale, timeunit)
        
    def reset(self, scale=1000, timeunit='ms'):
        self.scale = scale
        self.timeunit = timeunit
        self.name_list = []
        self.ts_dict = {}
        self.time_dict = collections.defaultdict(lambda: 0)
        self.min_time_dict = collections.defaultdict(lambda: 1e10)
        self.max_time_dict = collections.defaultdict(lambda: 0)
        self.count_dict = collections.defaultdict(lambda: 0)
        self.timelist_dict = collections.defaultdict(list)
        self.switch(True)
        
    def switch(self, onoff):
        self.__on = onoff
    
    def tic(self, name):
        if self.__on:
            if name not in self.name_list:
                self.name_list.append(name)
            self.ts_dict[name] = time.time()
        
    def toc(self, name, stack=False):
        if self.__on:
            dt = (time.time() - self.ts_dict[name]) * self.scale
            self.time_dict[name] = self.time_dict[name] + dt
            self.min_time_dict[name] = min(self.min_time_dict[name], dt)
            self.max_time_dict[name] = max(self.max_time_dict[name], dt)
            self.count_dict[name] = self.count_dict[name] + 1
            if stack:
                self.timelist_dict[name].append(dt)
            return dt
            
    def toctic(self, name_toc, name_tic, stack=False):
        dt = self.toc(name_toc, stack=stack)
        self.tic(name_tic)
        return dt
        
    def print_time_log(self, names=None):
        if names is None:
            names = self.name_list
        for name in names:
            print("{name}: \t{tot_T} {timeunit}/{tot_C} = {per_T} {timeunit} ({minT}/{maxT})\n".format(
                name=name, tot_T=np.round(np.sum(self.time_dict[name])), tot_C=self.count_dict[name], 
                per_T= np.round(np.sum(self.time_dict[name])/self.count_dict[name], 3),
                timeunit=self.timeunit, minT=round(self.min_time_dict[name],3), maxT=round(self.max_time_dict[name],3)
            ))
            
    def __str__(self):
        strout = "" 
        names = self.name_list
        for name in names:
            strout += "{name}: \t{tot_T} {timeunit}/{tot_C} = {per_T} {timeunit} ({minT}/{maxT})\n".format(
                name=name, tot_T=np.round(np.sum(self.time_dict[name])), tot_C=self.count_dict[name], 
                per_T= np.round(np.sum(self.time_dict[name])/self.count_dict[name], 3),
                timeunit=self.timeunit, minT=round(self.min_time_dict[name],3), maxT=round(self.max_time_dict[name],3)
            )
        return strout

def load_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data

def get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION):
    action_data_list = load_pickle(os.path.join(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION.replace("json", "pkl")))
    return len(action_data_list)

def load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, idx_act, joint_num):

    N_vtx_box = 3*8
    N_mask_box = 1
    N_joint_box = joint_num
    N_label_box = N_vtx_box+N_mask_box+N_joint_box
    N_vtx_cyl = 3*2+1
    N_mask_cyl = 1
    N_joint_cyl = joint_num
    N_label_cyl = N_vtx_cyl+N_mask_cyl+N_joint_cyl
    N_vtx_init = 3*8
    N_mask_init = 1
    N_joint_init = joint_num
    N_label_init = N_vtx_init+N_mask_init+N_joint_init
    N_vtx_goal = 3*8
    N_mask_goal = 1
    N_joint_goal = joint_num
    N_label_goal = N_vtx_goal+N_mask_goal+N_joint_goal
    N_joint_label = 6*joint_num
    N_cell_label = N_label_box+N_label_cyl+N_label_init+N_label_goal + N_joint_label
    N_BEGIN_CYL = N_vtx_box+N_mask_box+N_joint_box
    N_BEGIN_INIT = N_BEGIN_CYL+N_vtx_cyl+N_mask_cyl+N_joint_cyl
    N_BEGIN_GOAL = N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init

    scene_pickle = load_pickle(os.path.join(CONVERTED_PATH, DATASET, WORLD, SCENE, "scene.pkl"))
    scene_data = scene_pickle[b'scene_data']
    ctem_names = scene_pickle[b'ctem_names']
    ctem_cells = scene_pickle[b'ctem_cells']

    action_data_list = load_pickle(os.path.join(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION.replace("json", "pkl")))
    act_dat = action_data_list[idx_act]
    init_box_dat = act_dat[b'init_box_dat']
    goal_box_dat = act_dat[b'goal_box_dat']
    ctem_dat_list = act_dat[b'ctem_dat_list']
    skey = int(act_dat[b'skey'])
    success = act_dat[b'success']
    ### put init, goal item data
    cell, verts, chain = init_box_dat
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_INIT:N_BEGIN_INIT+N_vtx_init] = verts
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_INIT+N_vtx_init:N_BEGIN_INIT+N_vtx_init+N_mask_init] = 1
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_INIT+N_vtx_init+N_mask_init:N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init] = chain

    cell, verts, chain = goal_box_dat
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_GOAL:N_BEGIN_GOAL+N_vtx_goal] = verts
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_GOAL+N_vtx_goal:N_BEGIN_GOAL+N_vtx_goal+N_mask_goal] = 1
    scene_data[cell[0],cell[1],cell[2],N_BEGIN_GOAL+N_vtx_goal+N_mask_goal:N_BEGIN_GOAL+N_vtx_goal+N_mask_goal+N_joint_goal] = chain

    ### add/replace collilsion object
    for cname, ctype, cell, verts, chain in ctem_dat_list:
        if ctype == b'BOX':
            N_BEGIN_REP, N_vtx, N_mask, N_joint = 0, N_vtx_box, N_mask_box, N_joint_box
        elif ctype == b'CYLINDER':
            N_BEGIN_REP, N_vtx, N_mask, N_joint = N_BEGIN_CYL, N_vtx_cyl, N_mask_cyl, N_joint_cyl
        else:
            raise(RuntimeError("Non considered shape key"))
        scene_data[cell[0],cell[1],cell[2],N_BEGIN_REP:N_BEGIN_REP+N_vtx] = verts
        scene_data[cell[0],cell[1],cell[2],N_BEGIN_REP+N_vtx:N_BEGIN_REP+N_vtx+N_mask] = 1
        scene_data[cell[0],cell[1],cell[2],N_BEGIN_REP+N_vtx+N_mask:N_BEGIN_REP+N_vtx+N_mask+N_joint] = chain
    return scene_data, success, skey

In [2]:
CONVERTED_PATH = "./data/converted"
SCENE_FILENAME = "scene.pkl"
JOINT_NUM = 13
gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(filter(lambda x: x not in ["backup", "converted"], os.listdir(CONVERTED_PATH)))
DATASET = DATASET_LIST[0]
CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
#Iterate world
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
gtimer.reset()

In [3]:
SCENE_TUPLE_LIST = []
for WORLD in WORLD_LIST:
    WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
    # Iterate scene
    SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
    for SCENE in SCENE_LIST:
        SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
        ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
        for ACTION in ACTION_LIST:
            N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
            for i_act in range(N_action):
                SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                gtimer.tic("load_scene_data")
                scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                gtimer.toc("load_scene_data")
print(gtimer)

load_scene_data: 	6023.0 ms/6276 = 0.96 ms (0.659/6.082)



In [4]:
TRAIN_RATIO = 0.7
import random
N_train  = int(len(SCENE_TUPLE_LIST)*TRAIN_RATIO)
N_test = len(SCENE_TUPLE_LIST) - N_train
train_set = SCENE_TUPLE_LIST[:N_train]
test_set = SCENE_TUPLE_LIST[N_train:]

In [5]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPool3D
from tensorflow.keras import Model

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv3D(256, 3, activation='relu', padding='same') # 15
        self.conv2 = Conv3D(256, 3, activation='relu', padding='same')
        self.pool1 = MaxPool3D(pool_size=(3, 3, 3), strides=(2,2,2), padding='valid') # 7
        self.conv3 = Conv3D(256, 3, activation='relu', padding='same')
        self.conv4 = Conv3D(256, 3, activation='relu', padding='same')
        self.pool2 = MaxPool3D(pool_size=(3, 3, 3), strides=(2,2,2), padding='valid') # 3
        self.conv5 = Conv3D(256, 3, activation='relu', padding='same')
        self.conv6 = Conv3D(256, 3, activation='relu', padding='same')
        self.pool3 = MaxPool3D(pool_size=(3, 3, 3), strides=(2,2,2), padding='valid') # 1
        self.flatten = Flatten()
        self.d1 = Dense(256, activation='relu')
        self.d2 = Dense(128, activation='relu')
        self.d3 = Dense(64)
        self.d4 = Dense(2, activation='sigmoid')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        return self.d4(x)

# Create an instance of the model
model = MyModel()

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [10]:
EPOCHS = 5
BATCH_SIZE = 32
LOG_STEP = 100

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(train_set)
    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in train_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            train_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')

    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in test_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            test_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.6668033599853516, Accuracy: 64.71259307861328, Test Loss: 0.6591668128967285, Test Accuracy: 65.40948486328125
Epoch 2, Loss: 0.6654521822929382, Accuracy: 64.78102111816406, Test Loss: 0.6591668128967285, Test Accuracy: 65.40948486328125
Epoch 3, Loss: 0.6654521822929382, Accuracy: 64.78102111816406, Test Loss: 0.6591668128967285, Test Accuracy: 65.40948486328125
Epoch 4, Loss: 0.6661364436149597, Accuracy: 64.71259307861328, Test Loss: 0.6591668128967285, Test Accuracy: 65.40948486328125
Epoch 5, Loss: 0.6652240753173828, Accuracy: 64.8038330078125, Test Loss: 0.6591668128967285, Test Accuracy: 65.40948486328125


In [15]:
np.array(success_batch,dtype=np.int)

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0])

In [13]:
predictions = model(np.array(scene_batch, dtype=np.float32), training=False)

In [37]:
dtest = Dense(1, activation='sigmoid')

In [47]:
dtest((tf.constant([[10]])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[3.248686e-05]], dtype=float32)>

In [14]:
predictions

<tf.Tensor: shape=(27, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.14766165614128113, Accuracy: 95.56000518798828, Test Loss: 0.06508717685937881, Test Accuracy: 97.79999542236328
Epoch 2, Loss: 0.04511237516999245, Accuracy: 98.6500015258789, Test Loss: 0.05184425413608551, Test Accuracy: 98.25999450683594
Epoch 3, Loss: 0.023387636989355087, Accuracy: 99.20832824707031, Test Loss: 0.058742888271808624, Test Accuracy: 98.18999481201172
Epoch 4, Loss: 0.015396256931126118, Accuracy: 99.48999786376953, Test Loss: 0.04931311309337616, Test Accuracy: 98.48999786376953
Epoch 5, Loss: 0.009212623350322247, Accuracy: 99.70832824707031, Test Loss: 0.05627923458814621, Test Accuracy: 98.40999603271484


In [10]:
print(images.shape, labels.shape)

(32, 28, 28, 1) (32,)


In [11]:
labels

<tf.Tensor: shape=(32,), dtype=uint8, numpy=
array([3, 4, 6, 2, 3, 4, 8, 6, 4, 2, 6, 3, 9, 0, 1, 0, 8, 3, 6, 0, 8, 0,
       2, 2, 2, 0, 2, 2, 1, 3, 3, 7], dtype=uint8)>